In [ ]:
#import necessary libraries
import cv2
import torch
import time
import PIL.Image
from torchvision import transforms

In [ ]:
def real_time_evaluation(model, transform, device='cpu'):
    # Initialize video capture
    cap = cv2.VideoCapture(0)

    # Set camera resolution to 720p
    cap.set(3, 1280)  # Width
    cap.set(4, 720)   # Height

    # Initialize FPS counter
    prev_frame_time = 0
    new_frame_time = 0

    model.eval()  # Set model to evaluation mode

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Calculate FPS
        new_frame_time = time.time()
        fps = int(1/(new_frame_time-prev_frame_time))
        prev_frame_time = new_frame_time

        # Preprocess frame
        image = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        image = transform(image)
        image = image.unsqueeze(0).to(device)

        # Get prediction
        with torch.no_grad():
            prediction = model(image)
            prob = torch.nn.functional.softmax(prediction, dim=1)
            score, pred_idx = torch.max(prob, 1)

        # Display results on frame
        cv2.putText(frame, f'Class: {pred_idx.item()}', (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'Score: {score.item():.2f}', (30, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame, f'FPS: {fps}', (30, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display frame in Jupyter notebook
        cv2.imshow('Real-time Evaluation', frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Define your transform
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

In [ ]:
class TrafficSignNet(nn.Module):
    def __init__(self):
        super(TrafficSignNet, self).__init__()

        # First convolutional block
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Second convolutional block
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=5, padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Third convolutional block
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(2048, 43)
        )

    def forward(self, x):
        #x = x.view(x.size(0), -1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc(x)
        return x

In [ ]:
model = TrafficSignNet().to(device)
model.load_state_dict(torch.load('C:\Users\ARUSHI\OneDrive\Desktop\helios\traffic_sign_detection.pt'))

In [ ]:
# Call the function with your model
real_time_evaluation(model, transform)